# Multi-Agents based Systems and Genetic Algorithms

## Question

Questions répondues :

---

      1 . pourquoi dans ton polynôme , les valeurs low et up ne sont pas les mêmes que pour att ?
      A priori on mute dans le domaine de définition des attributs ? 
      donc ce que tu mets dans "att" et dans low/up devrait être la même chose ?
      
      réponse 1 . Correct ! J'avais mal appréhender la fonction de mutation !
      
---

      2 . quand on fait tourner selBest, en résultat il affiche pour chaque
      génération une colonne nevals. A quoi cela correspond ?
          
      
      réponse 2 . nevals = nombre d'évaluations d'individu (permet de voir combien ont changé)
      
---

      3 . - quand on fait du cross-over, que deviennent les parents ? est-ce que
            c'est la même chose avec selBest et SelTournanent ?
            
      réponse 3 .
    
---

Questions non-répondues :


- différence entre initRepeat et initIterate (que tu mets dans ton
voyageur de commerce) ?

- est-ce que la fonction select est prise en compte dans selBest ?
   (tu disais que non, mais tu avais aussi une erreur de code ... du
coup quelle est la conclusion ?)

- Si la réponse à la question précédente est "non" : quel algo conserve
les meilleurs ?

- Comment fonctionne selTournament ? je n'ai toujours pas compris



## Réponse

- Correct ! J'avais mal appréhender la fonction de mutation !

- Différence minime :
    * avec repeat : On va appeler n fois une fonction, à chaque fois le résultat est mis dans la structure de donnée qui est ici l'individu
    * avec iterate : Appelle une fois une fonction qui retourne un itérable, puis tout ce qu'il y a dans l'itérable sera mis dans l'individu
    
- Fonction de fitness plutot ? Et c'est l'attribut Fitness qui est prit en compte (code en dessous)

- A besoin de la réponse juste avant

- nevals = nombre d'évaluations d'individu (permet de voir combien ont changé)

- selTournament code source juste en bas

- les parents changent pour devenir les enfants (code en dessous)  Il n'y a pas de lien entre le select et le crossover donc à priori le comportement sera le même avec selBest et selTournament


- Lien utile : https://github.com/DEAP/deap/tree/master/deap

In [ ]:
def selRandom(individuals, k):
    """Select *k* individuals at random from the input *individuals* with
    replacement. The list returned contains references to the input
    *individuals*.
    :param individuals: A list of individuals to select from.
    :param k: The number of individuals to select.
    :returns: A list of selected individuals.
    This function uses the :func:`~random.choice` function from the
    python base :mod:`random` module.
    """
    return [random.choice(individuals) for i in xrange(k)]

def selTournament(individuals, k, tournsize, fit_attr="fitness"):
    """Select the best individual among *tournsize* randomly chosen
    individuals, *k* times. The list returned contains
    references to the input *individuals*.
    :param individuals: A list of individuals to select from.
    :param k: The number of individuals to select.
    :param tournsize: The number of individuals participating in each tournament.
    :param fit_attr: The attribute of individuals to use as selection criterion
    :returns: A list of selected individuals.
    This function uses the :func:`~random.choice` function from the python base
    :mod:`random` module.
    """
    chosen = []
    for i in xrange(k):
        aspirants = selRandom(individuals, tournsize)
        chosen.append(max(aspirants, key=attrgetter(fit_attr)))
    return chosen

In [ ]:
def selBest(individuals, k, fit_attr="fitness"):
    """Select the *k* best individuals among the input *individuals*. The
    list returned contains references to the input *individuals*.
    :param individuals: A list of individuals to select from.
    :param k: The number of individuals to select.
    :param fit_attr: The attribute of individuals to use as selection criterion
    :returns: A list containing the k best individuals.
    """
    return sorted(individuals, key=attrgetter(fit_attr), reverse=True)[:k]

In [ ]:
def cxOnePoint(ind1, ind2):
    """Executes a one point crossover on the input :term:`sequence` individuals.
    The two individuals are modified in place. The resulting individuals will
    respectively have the length of the other.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :returns: A tuple of two individuals.
    This function uses the :func:`~random.randint` function from the
    python base :mod:`random` module.
    """
    size = min(len(ind1), len(ind2))
    cxpoint = random.randint(1, size - 1)
    ind1[cxpoint:], ind2[cxpoint:] = ind2[cxpoint:], ind1[cxpoint:]

    return ind1, ind2

In [ ]:
def mutShuffleIndexes(individual, indpb):
    """Shuffle the attributes of the input individual and return the mutant.
    The *individual* is expected to be a :term:`sequence`. The *indpb* argument is the
    probability of each attribute to be moved. Usually this mutation is applied on
    vector of indices.
    :param individual: Individual to be mutated.
    :param indpb: Independent probability for each attribute to be exchanged to
                  another position.
    :returns: A tuple of one individual.
    This function uses the :func:`~random.random` and :func:`~random.randint`
    functions from the python base :mod:`random` module.
    """
    size = len(individual)
    for i in xrange(size):
        if random.random() < indpb:
            swap_indx = random.randint(0, size - 2)
            if swap_indx >= i:
                swap_indx += 1
            individual[i], individual[swap_indx] = \
                individual[swap_indx], individual[i]

    return individual,

def mutUniformInt(individual, low, up, indpb):
    """Mutate an individual by replacing attributes, with probability *indpb*,
    by a integer uniformly drawn between *low* and *up* inclusively.
    :param individual: :term:`Sequence <sequence>` individual to be mutated.
    :param low: The lower bound or a :term:`python:sequence` of
                of lower bounds of the range from which to draw the new
                integer.
    :param up: The upper bound or a :term:`python:sequence` of
               of upper bounds of the range from which to draw the new
               integer.
    :param indpb: Independent probability for each attribute to be mutated.
    :returns: A tuple of one individual.
    """
    size = len(individual)
    if not isinstance(low, Sequence):
        low = repeat(low, size)
    elif len(low) < size:
        raise IndexError("low must be at least the size of individual: %d < %d" % (len(low), size))
    if not isinstance(up, Sequence):
        up = repeat(up, size)
    elif len(up) < size:
        raise IndexError("up must be at least the size of individual: %d < %d" % (len(up), size))

    for i, xl, xu in zip(xrange(size), low, up):
        if random.random() < indpb:
            individual[i] = random.randint(xl, xu)

    return individual,

## SMA simple avec Algo gen

### SMA

In [5]:
import numpy as np
import networkx as nx
import random
import matplotlib.pyplot as plt

def isPrime(n):
    if n > 1:
        for i in range(2, n):
            if (n % i) == 0:
                return False
        return True
    else:
        return False

class Agent:
    
    def __init__(self, sma, id_number, accointances, color): 
        self.id_number = id_number
        self.accointances = accointances
        self.sma = sma
        self.color = color
        self.ball = None
        self.stuck = False
        self.nbGetBall = 0
            
    def decide(self, tick):
        if self.ball != None:
            possibleGive = []
            voisins = [self.sma.getAgent(i) for i in self.accointances]
            for voisin in voisins:
                if voisin.id_number % self.color == 0:
                    possibleGive += [voisin]
                elif self.color == 2 and isPrime(voisin.id_number):
                    possibleGive += [voisin]
                elif self.color == 3 and voisin.id_number % 2 == 1:
                    possibleGive += [voisin]
            if len(possibleGive) != 0:
                self.giveBallTo(random.choice(possibleGive))
            else:
                self.stuck = True
            
    def getTheBall(self,ball):
        #print(self)
        self.ball = ball
        self.nbGetBall += 1
        
    def giveBallTo(self,agent):
        ball = self.ball
        agent.getTheBall(ball)
        self.ball = None
        
    def __str__(self):
        return "- Agent "+str(self.id_number)+" -"
    
    def __repr__(self):
        return str(self)
    
COLORS = [2,3,5,11]
BALL = "je suis une balle"
    
class SMA:
    def __init__(self, nb_agents,graph,adjacency_matrix,perColor1,perColor2,perColor3,perColor4):
        self.tick = 0
        self.nb_agents = nb_agents
        self.agentList = []
        self.stuck = False
        self.graph = graph
        perColors = [perColor1,perColor2,perColor3,perColor4]
        
        # Initialize accointances based on the matrix
        for i in range(nb_agents):
            self.agentList.append(Agent(self, random.randint(0,nb_agents), np.nonzero(adjacency_matrix[i])[0],random.choices(COLORS, weights=perColors, k=1)[0]))
    
    def createSMA_graph(graph,perColor1,perColor2,perColor3,perColor4):
        matrix = nx.to_numpy_array(graph, nodelist=range(graph.number_of_nodes()))
        return SMA(graph.number_of_nodes(),graph, matrix, perColor1,perColor2,perColor3,perColor4)
    
    def createSMA_nbagents(nb_agents,nb_contact,perColor1,perColor2,perColor3,perColor4):
        graph = nx.barabasi_albert_graph(nb_agents, nb_contact)
        return SMA.createSMA_graph(graph,perColor1,perColor2,perColor3,perColor4)
    
    def run(self, rounds):
        self.tick = 0
        self.clearAll()
        self.setBall(random.randint(0,len(self.agentList)-1))
        while(not(self.isStuck()) and self.tick<rounds and not(self.isFinished())):
            self.runOnce()
        #self.whoHasTheBall()
        #self.show()
        if self.isStuck():
            return rounds * 2
            #return rounds
        elif self.isFinished():
            #print("yeah !")
            return self.tick
        else:
            return self.tick
            
    def runOnce(self):
        self.tick += 1
        for agent in self.agentList:
            agent.decide(self.tick)
        #plt.close()
        #self.show()
            
    def getAgent(self,i):
        return self.agentList[i]
    
    def isStuck(self):
        return any([agent.stuck for agent in self.agentList])
    
    def isFinished(self):
        return all([(agent.nbGetBall > 0) for agent in self.agentList])
    
    def setBall(self,i):
        self.getAgent(i).getTheBall(BALL)
         
    def clear(self,i):
        agent = self.getAgent(i)
        agent.ball = None
        agent.stuck = False
        
    def clearAll(self):
        for i in range(len(self.agentList)):
            self.clear(i)
            
    def whoHasTheBall(self):
        for agent in self.agentList:
            if agent.ball != None:
                print(agent)
            
    def show(self):
        plt.figure(figsize=(10,5))
        colors = []
        for agent in self.agentList:
            if agent.color == 2:
                colors += ["green"]
            elif agent.color == 3:
                colors += ["blue"]
            elif agent.color == 5:
                colors += ["orange"]
            else:
                colors += ["red"]
        nx.draw_networkx(self.graph, node_color=colors)
        plt.axis('off')
        plt.show()
            

In [6]:
N = 10
res = []
for _ in range(100):
    sma = SMA.createSMA_nbagents(N,3,60,30,10,0)
    res += [sma.run(100)]
np.mean(res)

98.39

In [4]:
from deap import creator, tools, base, algorithms

def evaluate_SMA(N,ind):
    res = []
    for i in range(10):
        sma = SMA.createSMA_nbagents(N,ind[0],ind[1],ind[2],ind[3],ind[4])
        res += [sma.run(100)]
    return (np.mean(res),)
    

def algo_gen_SMA(nbind_pop,nb_gen,nb_cst,max_val_cst):
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    
    toolbox=base.Toolbox()
    toolbox.register("att",random.randint,1,max_val_cst)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.att, nb_cst)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate",evaluate_SMA,nbind_pop)
    toolbox.register("mutate",tools.mutUniformInt,low=1,up=max_val_cst,indpb=0.2)
    toolbox.register("mate",tools.cxOnePoint)
    toolbox.register("select",tools.selTournament,tournsize=3)
    
    hof = tools.HallOfFame(1)

    stats = tools.Statistics(lambda ind : ind.fitness.values)
    stats.register("avg",np.mean)
    stats.register("std",np.std)
    stats.register("min",np.min)
    stats.register("max",np.max)
    
    pop = toolbox.population(n=nbind_pop)
    
    pop, log = algorithms.eaSimple(pop,toolbox,cxpb=0.5, mutpb=0.2 , ngen=nb_gen , stats=stats, halloffame=hof, verbose=False)
    
    return {"pop":pop,"log":log,"hof":hof,"stats": stats}

In [10]:
res = algo_gen_SMA(10,100,5,9)

In [7]:
res["hof"][0]

[3, 6, 8, 1, 2]

In [9]:
res["hof"][0]

[6, 9, 9, 5, 1]

In [11]:
res["hof"][0]

[5, 7, 9, 1, 1]